In [13]:
import pandas as pd
import numpy as np
from sklearn.metrics import cohen_kappa_score
from statsmodels.stats.inter_rater import fleiss_kappa
import krippendorff
from sklearn.preprocessing import LabelEncoder

In [3]:
data = pd.read_csv('../data/merged_annotations.csv')

In [4]:
len(data)

82

In [48]:
columns = ['inclusion', 'phenomenon_short', 'phenomenon_defined', 'definition_scope', 'definition_integrity',
           'definition_integrity_detail', 'task_face_validity', 
           'task_train_val', 'response_format', 'metric_access', 'metric_face_validity', 
           'result_interpretation', 'results_comparison', 'results_comparison_explanation', 
           'results_human_baseline', 'results_author_validity', 'results_realism', 'authorship', 
           'benchmark_availability']

## Cohen's Kappa

In [49]:
def compute_kappa_per_feature(df, sample_id_col, feature_cols):
    """
    Computes Cohen's Kappa for each feature where two annotations are available per sample.

    Args:
        df (pd.DataFrame): DataFrame containing all annotations.
        sample_id_col (str): Column name for the sample ID.
        feature_cols (list): List of column names (features) to compute agreement on.

    Returns:
        pd.Series: Cohen's Kappa scores per feature.
    """
    kappas = {}

    # Group by sample_id to get annotation pairs
    grouped = df.groupby(sample_id_col)

    # Only keep samples with exactly 2 annotations
    valid_groups = {k: g for k, g in grouped if len(g) == 2}

    for feature in feature_cols:
        ann1 = []
        ann2 = []

        for g in valid_groups.values():
            # Get the 2 values for this feature
            values = g[feature].values
            if len(values) == 2:
                ann1.append(values[0])
                ann2.append(values[1])

        if ann1 and ann2:
            kappa = cohen_kappa_score(ann1, ann2)
            kappas[feature] = kappa
        else:
            kappas[feature] = None  # or np.nan

    return pd.Series(kappas, name='cohen_kappa')


In [50]:
# Define your ID column and feature columns
sample_id_col = 'bibkey'
feature_cols = columns

# Compute Kappa per feature
kappa_scores = compute_kappa_per_feature(data, sample_id_col, feature_cols)

# Display the results
print(kappa_scores)

# Optional: compute mean kappa across all features
mean_kappa = kappa_scores.mean()
print(f"Mean Cohen's Kappa: {mean_kappa:.3f}")

inclusion                         0.000000
phenomenon_short                  0.163265
phenomenon_defined               -0.134163
phenomenon_contested              0.000000
definition_scope                  0.001282
definition_integrity              0.126627
definition_integrity_detail       0.210646
task_face_validity                0.484277
task_train_val                    0.493306
response_format                   0.447439
metric_access                     0.372449
metric_face_validity             -0.051282
result_interpretation            -0.037975
results_comparison                0.196078
results_comparison_explanation    0.189386
results_human_baseline            0.583991
results_author_validity           0.154639
results_realism                   0.095588
authorship                        0.515748
benchmark_availability            0.000000
Name: cohen_kappa, dtype: float64
Mean Cohen's Kappa: 0.191


## Krippendorff

In [51]:
def compute_krippendorffs_alpha_encoded(df, sample_id_col, feature_cols):
    """
    Computes Krippendorff's Alpha (nominal) per feature after encoding string categories.

    Args:
        df (pd.DataFrame): Annotations dataframe with sample IDs and categorical features.
        sample_id_col (str): Column name indicating sample ID.
        feature_cols (list): Feature names (categorical).

    Returns:
        pd.Series: Krippendorff's alpha values per feature.
    """
    alpha_scores = {}

    grouped = df.groupby(sample_id_col)

    for feature in feature_cols:
        # Encode string categories to integers
        le = LabelEncoder()
        try:
            encoded_feature = le.fit_transform(df[feature].astype(str))
        except Exception:
            alpha_scores[feature] = np.nan
            continue

        df_encoded = df.copy()
        df_encoded[feature] = encoded_feature

        data = []

        for _, group in df_encoded.groupby(sample_id_col):
            values = group[feature].values
            if len(values) == 2:
                data.append(values)
            elif len(values) == 1:
                data.append([values[0], np.nan])
            else:
                continue  # skip samples with >2 annotations

        if not data:
            alpha_scores[feature] = np.nan
            continue

        matrix = np.array(data).T  # rows = annotators, columns = items
        try:
            alpha = krippendorff.alpha(reliability_data=matrix, level_of_measurement='nominal')
            alpha_scores[feature] = alpha
        except Exception:
            alpha_scores[feature] = np.nan

    return pd.Series(alpha_scores, name='krippendorffs_alpha')


In [52]:
sample_id_col = 'bibkey'
feature_cols = columns

alpha_scores = compute_krippendorffs_alpha_encoded(data, sample_id_col, feature_cols)

print(alpha_scores)
print(f"Mean Krippendorff’s Alpha: {alpha_scores.mean():.3f}")

inclusion                         0.000000
phenomenon_short                  0.162409
phenomenon_defined               -0.144876
phenomenon_contested             -0.295242
definition_scope                 -0.002606
definition_integrity              0.132143
definition_integrity_detail       0.217169
task_face_validity                0.488959
task_train_val                    0.496112
response_format                   0.449541
metric_access                     0.375321
metric_face_validity             -0.040685
result_interpretation            -0.028571
results_comparison                0.185689
results_comparison_explanation    0.183554
results_human_baseline            0.587436
results_author_validity           0.118607
results_realism                   0.063584
authorship                        0.521182
benchmark_availability           -0.016736
Name: krippendorffs_alpha, dtype: float64
Mean Krippendorff’s Alpha: 0.173


## Gwet’s AC1

In [53]:
def compute_gwets_ac1(df, sample_id_col, feature_cols):
    """
    Computes Gwet's AC1 per feature assuming 2 annotations per sample (nominal data).

    Args:
        df (pd.DataFrame): DataFrame with categorical annotations and sample IDs.
        sample_id_col (str): Column with sample ID.
        feature_cols (list): Categorical feature columns.

    Returns:
        pd.Series: Gwet's AC1 per feature.
    """
    ac1_scores = {}

    grouped = df.groupby(sample_id_col)

    for feature in feature_cols:
        # Extract annotation pairs
        pairs = []
        for _, group in grouped:
            values = group[feature].dropna().astype(str).values
            if len(values) == 2:
                pairs.append(values)

        if not pairs:
            ac1_scores[feature] = np.nan
            continue

        # Flatten and encode to integers
        all_vals = np.array(pairs).flatten()
        le = LabelEncoder()
        all_vals_encoded = le.fit_transform(all_vals)

        # Re-encode pairs with same mapping
        encoded_pairs = all_vals_encoded.reshape(len(pairs), 2)

        # Agreement proportion
        agree = np.sum(encoded_pairs[:, 0] == encoded_pairs[:, 1])
        n = len(encoded_pairs)
        p_o = agree / n

        # Category proportions
        num_categories = len(le.classes_)
        cat_counts = np.zeros(num_categories)
        for i in range(num_categories):
            cat_counts[i] = np.sum(encoded_pairs == i)

        cat_probs = cat_counts / (2 * n)

        # Expected agreement (Pe) under Gwet’s formulation
        p_e = np.sum(cat_probs * cat_probs)

        # AC1 formula
        if p_e != 1:
            ac1 = (p_o - p_e) / (1 - p_e)
        else:
            ac1 = 1.0

        ac1_scores[feature] = ac1

    return pd.Series(ac1_scores, name='gwets_ac1')

In [54]:
ac1_scores = compute_gwets_ac1(data, sample_id_col, feature_cols)

print(ac1_scores)
print(f"Mean Gwet’s AC1: {ac1_scores.mean():.3f}")

inclusion                        -0.012346
phenomenon_short                  0.151756
phenomenon_defined               -0.159420
phenomenon_contested             -0.319588
definition_scope                 -0.000755
definition_integrity              0.132100
definition_integrity_detail       0.240753
task_face_validity                0.656652
task_train_val                    0.534606
response_format                   0.456949
metric_access                     0.473684
metric_face_validity             -0.043478
result_interpretation            -0.038961
results_comparison                0.181586
results_comparison_explanation    0.191956
results_human_baseline            0.680672
results_author_validity           0.120301
results_realism                   0.063361
authorship                        0.539026
benchmark_availability           -0.025641
Name: gwets_ac1, dtype: float64
Mean Gwet’s AC1: 0.191


## Percent Agreement per Feature

In [55]:
def compute_percent_agreement(df, sample_id_col, feature_cols):
    """
    Computes raw agreement (percent of exact matches) per feature,
    assuming 2 annotations per sample and no explicit annotator IDs.

    Args:
        df (pd.DataFrame): Annotations DataFrame with sample IDs and features.
        sample_id_col (str): Column name identifying the sample.
        feature_cols (list): List of categorical feature column names.

    Returns:
        pd.Series: Percent agreement (0–1) per feature.
    """
    agreement_scores = {}

    grouped = df.groupby(sample_id_col)

    for feature in feature_cols:
        matches = []
        for _, group in grouped:
            values = group[feature].dropna().values
            if len(values) == 2:
                matches.append(values[0] == values[1])
        if matches:
            agreement_scores[feature] = np.mean(matches)
        else:
            agreement_scores[feature] = np.nan  # no valid pairs

    return pd.Series(agreement_scores, name='percent_agreement')

In [56]:
percent_agreement = compute_percent_agreement(data, sample_id_col, feature_cols)

print(percent_agreement)
print(f"Mean Percent Agreement: {percent_agreement.mean():.3f}")

inclusion                         0.975610
phenomenon_short                  0.600000
phenomenon_defined                0.525000
phenomenon_contested              0.000000
definition_scope                  0.564103
definition_integrity              0.575000
definition_integrity_detail       0.615385
task_face_validity                0.975000
task_train_val                    0.743590
response_format                   0.525000
metric_access                     0.950000
metric_face_validity              0.897436
result_interpretation             0.925000
results_comparison                0.600000
results_comparison_explanation    0.564103
results_human_baseline            0.842105
results_author_validity           0.538462
results_realism                   0.575000
authorship                        0.725000
benchmark_availability            0.950000
Name: percent_agreement, dtype: float64
Mean Percent Agreement: 0.683
